<a href="https://colab.research.google.com/github/KeoniM/NFL_Data_Cleaning/blob/main/NFL_Plays_Week2_2023_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PURPOSE:**
- Accurately clean a week's worth of play data
  - Season 2023 -> Week 2

**NOTE:**
- What makes version 2 different than version 1 is the data being used. Although the core of the data is identical to the original, NFL.com has updated their formatting of how they display their data which has been scraped and used here. So minor adjustments will have to be made in creating the new version but I also see a beautiful opportunity to clean the older version here. Make the code more readible, organized and efficient.

# MOUNTING AND IMPORTS

In [65]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
# Used to access personal google cloud services
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [67]:
# Imports

# Data manipulation
import pandas as pd

# Regular expressions
import re

# Database access
from google.cloud import bigquery

# LOADING DATA (BigQuery)

In [68]:
# Client connect to bigquery project
client = bigquery.Client('nfl-data-430702')

## Season 2023 Week 2

In [69]:
# Grabbing all plays from 2023 Week 2 NFL Sesason
nfl_plays_week2_2023_query = """
                             SELECT *
                             FROM `nfl-data-430702.NFL_Scores_v2.NFL-Plays-Week2_2023`
                             """

# Running psuedo query, and returns the amount of bytes it will take to run query
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(nfl_plays_week2_2023_query, job_config=dry_run_config)
print("This query will process {} gigabytes.".format(dry_run_query.total_bytes_processed/10**9))

# Running query (Being mindful of the amount of data being grabbed)
# Will grab a maximum of a Gigabyte
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(nfl_plays_week2_2023_query, job_config=safe_config)

This query will process 0.000645655 gigabytes.


In [70]:
# Putting data attained from query into a dataframe
week2_2023_plays = safe_config_query.to_dataframe()

In [71]:
week2_2023_plays.head()

Season    Week  Day   Date AwayTeam HomeTeam      Quarter  DriveNumber  \
0    2023  Week 2  THU  09/14  Vikings   Eagles  1st Quarter            1   
1    2023  Week 2  THU  09/14  Vikings   Eagles  1st Quarter            1   
2    2023  Week 2  THU  09/14  Vikings   Eagles  1st Quarter            1   
3    2023  Week 2  THU  09/14  Vikings   Eagles  1st Quarter            1   
4    2023  Week 2  THU  09/14  Vikings   Eagles  1st Quarter            1   

    TeamWithPossession  IsScoringDrive  PlayNumberInDrive  IsScoringPlay  \
0  Philadelphia Eagles               1                  1              0   
1  Philadelphia Eagles               1                  2              0   
2  Philadelphia Eagles               1                  3              0   
3  Philadelphia Eagles               1                  4              0   
4  Philadelphia Eagles               1                  5              0   

           PlayOutcome           PlayStart  PlayTimeFormation  \
0  Kickoff from MIN 35                None            Kickoff   
1          6 Yard Pass  1st & 10 at PHI 25  15:00 1st Shotgun   
2         -1 Yard Sack   2nd & 4 at PHI 31  14:27 1st Shotgun   
3           7 Yard Run   3rd & 5 at PHI 30  13:45 1st Shotgun   
4         -1 Yard Pass  1st & 10 at PHI 37  13:10 1st Shotgun   

                                     PlayDescription  
0  — G.Joseph kicks 65 yards from MIN 35 to end z...  
1  — J.Hurts pass short right to D.Smith to PHI 3...  
2  — J.Hurts sacked at PHI 30 for -1 yards (sack ...  
3  — J.Hurts scrambles right end pushed ob at PHI...  
4  — J.Hurts pass short left to D.Goedert to PHI ...

# CATEGORIZE PLAYS
- The goal here is to parse out the different values for 'PlayOutcome'
  - Here is where I will separate different types of plays
    - ( pass / run / kickoff / etc..)

In [72]:
# All play outcomes from the game
# - From here we can categorize and clean plays accordingly
week2_2023_plays['PlayOutcome'].unique()

array(['Kickoff from MIN 35', '6 Yard Pass', '-1 Yard Sack', '7 Yard Run',
       '-1 Yard Pass', '54 Yard Pass', '1 Yard Pass', '1 Yard Run',
       '2 Yard Run', 'Field Goal', 'Kickoff from PHI 35', '15 Yard Pass',
       'Pass Incomplete', 'Punt', '-5 Yard Penalty', '3 Yard Run',
       'Fumble', '4 Yard Run', '12 Yard Run', '-7 Yard Sack',
       'Interception', '0 Yard Run', '5 Yard Pass', '-3 Yard Run',
       'Field Goal No Good', '5 Yard Penalty', '9 Yard Pass',
       '-2 Yard Run', '3 Yard Pass', '24 Yard Pass', '5 Yard Run',
       '7 Yard Pass', 'Touchdown', 'Extra Point', '6 Yard Run',
       '8 Yard Run', '11 Yard Pass', 'Timeout', '13 Yard Pass',
       '4 Yard Pass', '18 Yard Pass', '18 Yard Run', '0 Yard Pass',
       '-5 Yard Pass', '-10 Yard Penalty', '2 Yard Pass', '9 Yard Run',
       '11 Yard Run', '8 Yard Pass', '-2 Yard Sack', '-12 Yard Sack',
       '23 Yard Pass', '22 Yard Pass', '14 Yard Pass', '12 Yard Pass',
       '43 Yard Run', '10 Yard Pass', '16 Yard Pa

In [73]:
# NOTES:
# - Currently, I am eyeing all unique play outcomes to categorizing them.
#   - This type of approach is not flexable because a play outcome can
#     arise that has not been seen yet.
#     - There may be more play outcomes in the future when working on a full season,
#       let alone all seasons and future games

# Play Types with complete cleaning methods (As far as this sample size goes)

# ~ OFFENSE ~
df_2023_pass_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Pass')]
df_2023_run_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Run')]
# ~ DEFENSE ~
df_2023_interception_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Interception')]
df_2023_sack_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Sack')]
# ~ SPECIAL TEAMS ~
df_2023_punt_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Punt')]
df_2023_kickoff_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Kickoff')]
# ~ SCORING ~
df_2023_touchdown_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Touchdown')]
df_2023_extrapoint_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Extra Point')]
df_2023_fieldgoal_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Field Goal')]
# df_2023_2pt_conversion_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('2PT Conversion')]
df_2023_2pt_conversion_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Conversion')]
# ~ OTHER ~
df_2023_fumble_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Fumble')]
df_2023_penalty_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Penalty')]
df_2023_turnover_on_downs_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Turnover on Downs')]
df_2023_timeout_week2 = week2_2023_plays[week2_2023_plays['PlayOutcome'].str.contains('Timeout')]

## SANITY CHECK (All Plays Accounted for)
  - Once all plays have been categorized, will compare the sum of all plays in each category to the size of the original dataframe of plays.
    - Goal is to make sure the number of plays is the same.

In [74]:
# Categorized plays

plays_list = [df_2023_pass_week2,         # Offense
              df_2023_run_week2,
              df_2023_interception_week2, # Defense
              df_2023_sack_week2,
              df_2023_punt_week2,         # Special Teams
              df_2023_kickoff_week2,
              df_2023_touchdown_week2,    # Scoring
              df_2023_extrapoint_week2,
              df_2023_fieldgoal_week2,
              df_2023_2pt_conversion_week2,
              df_2023_fumble_week2,       # Other
              df_2023_penalty_week2,
              df_2023_turnover_on_downs_week2,
              df_2023_timeout_week2]

num_plays_categorized = 0

for plays in plays_list:
  num_plays_categorized = num_plays_categorized + len(plays)

num_plays_categorized == len(week2_2023_plays)

True

# PIPELINE
- ORDER
  1. Team Dictionary
    - Used to map team names with their acronyms
  2. Regular expressions
    - Used to find common patterns within raw data
  3. Transforming Data
    - So far, only label encoding
  4. Cleaning methods
    - Unique cleaning methods for each play type
  5. Main pipeline method
    - Control flow of cleaning methods

## 1. TEAM DICTIONARY

In [75]:
# KEY: Team name
# VALUE: Acronym of team

dict_teams = {
    'Cardinals': 'ARI', 'Falcons': 'ATL', 'Ravens': 'BAL', 'Bills': 'BUF', 'Panthers': 'CAR', 'Bears': 'CHI',
    'Bengals': 'CIN', 'Browns': 'CLE', 'Cowboys': 'DAL', 'Broncos': 'DEN', 'Lions': 'DET', 'Packers': 'GB',
    'Texans': 'HOU', 'Colts': 'IND', 'Jaguars': 'JAX', 'Chiefs': 'KC', 'Raiders': 'LV', 'Chargers': 'LAC',
    'Rams': 'LAR', 'Dolphins': 'MIA', 'Vikings': 'MIN', 'Patriots': 'NE', 'Saints': 'NO', 'Giants': 'NYG',
    'Jets': 'NYJ', 'Eagles': 'PHI', 'Steelers': 'PIT', '49ers': 'SF', 'Seahawks': 'SEA', 'Buccaneers': 'TB',
    'Titans': 'TEN', 'Commanders': 'WAS'
}

In [76]:
# KEY: Full Team name
# VALUE: Acronym of team

dict_teams_2 = {
    'Arizona Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Ravens': 'BAL', 'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL', 'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX', 'Kansas City Chiefs': 'KC', 'Las Vegas Raiders': 'LV', 'Los Angeles Chargers': 'LAC',
    'Los Angeles Rams': 'LAR', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN', 'New England Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG',
    'New York Jets': 'NYJ', 'Philadelphia Eagles': 'PHI', 'Pittsburgh Steelers': 'PIT', 'San Francisco 49ers': 'SF', 'Seattle Seahawks': 'SEA', 'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN', 'Washington Commanders': 'WAS'
}

In [77]:
# KEY: Acronym of team
# VALUE: Team name

dict_teams_3 = {
    'ARI': 'Arizona Cardinals', 'ATL': 'Atlanta Falcons', 'BAL': 'Baltimore Ravens', 'BUF': 'Buffalo Bills', 'CAR': 'Carolina Panthers', 'CHI': 'Chicago Bears',
    'CIN': 'Cincinnati Bengals', 'CLE': 'Cleveland Browns', 'DAL': 'Dallas Cowboys', 'DEN': 'Denver Broncos', 'DET': 'Detroit Lions', 'GB': 'Green Bay Packers',
    'HOU': 'Houston Texans', 'IND': 'Indianapolis Colts', 'JAX': 'Jacksonville Jaguars', 'KC': 'Kansas City Chiefs', 'LV': 'Las Vegas Raiders', 'LAC': 'Los Angeles Chargers',
    'LAR': 'Los Angeles Rams', 'MIA': 'Miami Dolphins', 'MIN': 'Minnesota Vikings', 'NE': 'New England Patriots', 'NO': 'New Orleans Saints', 'NYG': 'New York Giants',
    'NYJ': 'New York Jets', 'PHI': 'Philadelphia Eagles', 'PIT': 'Pittsburgh Steelers', 'SF': 'San Francisco 49ers', 'SEA': 'Seattle Seahawks', 'TB': 'Tampa Bay Buccaneers',
    'TEN': 'Tennessee Titans', 'WAS': 'Washington Commanders'
}

## 2. REGULAR EXPRESSIONS

## 3. TRANSFORMING DATA

In [78]:
# TASKS:
# 1. Split 'PlayTimeFormation' feature into 3
#    1. TimeLeftInQuarter
#    2. Quarter (Replace current quarter value, the raw value is not 100% accurate)
#    3. PlayFormation
# 2. Split 'PlayStart' feature into 2
#    1. DownAndDistance
#    2. FieldPosition

In [79]:
# PURPOSE:
# - Take value for 'PlayTimeFormation' and split into 3 separate features.
#   1. TimeLeftInQuarter (Will come about when renaming 'PlayTimeFormation')
#   2. Quarter (This feature already exists, the values within 'PlayTimeFormation' are more accurate and will replace the value in here originaly)
#   3. PlayFormation

def playtimeformation_split(df_plays):

  df_plays_copy = df_plays.copy()

  new_columns = ['PlayFormation']

  df_plays_copy = df_plays_copy.rename(columns = {'PlayTimeFormation': 'TimeLeftInQuarter'})

  df_plays = df_plays.reindex(columns=df_plays.columns.tolist() + new_columns)

  # Splitting original feauture 'PlayTimeFormation' (Now known as 'TimeLeftInQuarter')
  for idx, play in df_plays_copy['TimeLeftInQuarter'].items():
    value_elements = play.split(' ')
    # Some plays (e.g. Kickoff) will only have the formation as a value
    if len(value_elements) <= 1:
      df_plays_copy.at[idx, 'PlayFormation'] = value_elements[0]
      df_plays_copy.at[idx, 'TimeLeftInQuarter'] = ""
    else:
      df_plays_copy.at[idx, 'TimeLeftInQuarter'] = value_elements[0]
      df_plays_copy.at[idx, 'Quarter'] = value_elements[1]
      df_plays_copy.at[idx, 'PlayFormation'] = " ".join(value_elements[2::])

  # Transform values in 'Quarter' feature from string to integer (e.g. '1st Quarter' -> 1)
  dict_replace_quarter = {'1st Quarter': 1, '2nd Quarter': 2, '3rd Quarter': 3, '4th Quarter': 4,
                          '1st': 1, '2nd': 2, '3rd': 3, '4th': 4}

  # All overtime quarters will be have the value 5 in their place
  df_plays_copy['Quarter'] = df_plays_copy['Quarter'].map(dict_replace_quarter).fillna(5).astype(int)

  return df_plays_copy

# PURPOSE:
# - Take value for 'PlayStart' and split into 2 separate features.
#   1. DownAndDistance (Will come about when renaming 'PlayStart')
#   2. FieldPosition (Start of play)

def playstart_split(df_plays):

  df_plays_copy = df_plays.copy()

  new_columns = ['FieldPosition']

  df_plays_copy = df_plays_copy.rename(columns = {'PlayStart': 'DownAndDistance'})

  df_plays_copy = df_plays_copy.reindex(columns=df_plays_copy.columns.tolist() + new_columns)

  df_plays_copy['FieldPosition'] = df_plays_copy['FieldPosition'].astype(str)

  # Splitting original feature 'PlayStart' (Now known as 'DownAndDistance')
  for idx, play in df_plays_copy['DownAndDistance'].items():
    # Some plays to not have a down and distance or field position and contain 'nan' values here,
    # this is to catcht those plays and keep going. (e.g. Kickoff / Extra Point / etc..)
    if pd.isna(play):
      continue
    else:
      value_elements = play.split(' at ')
      df_plays_copy.at[idx, 'DownAndDistance'] = value_elements[0]
      df_plays_copy.at[idx, 'FieldPosition'] = value_elements[1]

  return df_plays_copy

# PURPOSE:
# - Keep consistence with team names
#   - A team name will always be represented by their acronym

def consistent_team_names(df_plays):

  df_plays_copy = df_plays.copy()

  df_plays_copy['AwayTeam'] = df_plays_copy['AwayTeam'].map(dict_teams)
  df_plays_copy['HomeTeam'] = df_plays_copy['HomeTeam'].map(dict_teams)
  df_plays_copy['TeamWithPossession'] = df_plays_copy['TeamWithPossession'].map(dict_teams_2)

  return df_plays_copy

## 4. PIPELINE MAIN METHOD

In [80]:
# PURPOSE:
# - Accept a dataframe of nfl plays (formatted by NFL_Scrapers) and
#   return a cleaned dataframe of those plays.
# INPUT PARAMETERS:
# df_all_plays         - dataframe - all plays in raw form from NFL_Scraper that user
#                                    would like to clean.
# OUTPUT:
# df_all_plays_cleaned - dataframe - all plays from 'df_all_plays' cleaned and data
#                                    dispersed into individual new features.

# CURRENT DESIGN PLAN:
# 1. Use uniquely designed methods for each play type to clean within dataframe
#    - (e.g. pass, run, touchdown, punt, sack, ... )
# 2. Repeat until all plays within dataframe have been cleaned.
#   NOTE:
#   - It is important to fully clean a play type before moving to the next
#      because sometimes cleaning could involve adding a new row to the dataframe,
#      causing a reset to the dataframes indexing.
#      - If we were to separate all play types from the beginning, the indexes
#        could shift around causing, for example, an index that might originally
#        point to a run play to now instead point at a pass play.

def clean_dataframe_of_plays(df_all_plays):

  ################################
  # RAW DATA COLUMN DESCRIPTIONS #
  ################################

  # Season             - Year of the season
  # Week               - Game week of the season (e.g. 'Week 1')
  # Day                - Day of the week (e.g. 'MON')
  # Date               - Month and day of the game formatted MM/DD (e.g. '09/07')
  # AwayTeam           - Visiting team of the game
  # HomeTeam           - Home team of the game
  # Quarter            - Quarter that the play is in
  #                      - NOT ACCURATE. Drives that go between quarters will end up
  #                        having all plays in the later quarter.
  # DriveNumber        - Drive number of the quarter that the play is in
  # TeamWithPossession - Team that started with the ball at the beginning of the play.
  # IsScoringDrive     - Does the drive that the focused play in result in a score?
  # PlayNumberInDrive  - Play count in the drive
  # IsScoringPlay      - Did the play result in a score?
  # PlayOutcome        - Ultimate result of the play (e.g. '13 Yard Pass')
  # PlayStart          - The down and where the play started on the field (e.g. '2nd & 9 at DET 21')
  # PlayTimeFormation  - Time left in the quarter / quarter / play formation
  # PlayDescription    - The raw description given of the focused play, entailing everything
  #                      that happened within it.

  ######################################
  # NEW ADDITIONAL COLUMN DESCRIPTIONS #
  ######################################

  # ~ General features ~
  # TimeOnTheClock     - The time that was on the clock when the play started

  # ~ Offensive features ~
  # PlayType           - The type of play (e.g. pass/run)
  # Formation          - Play formation
  # Passer             - Player that threw the ball (mostly the quarterback)
  # Rusher             - Player that ran the ball (mostly the runningback)
  # Receiver           - Player on the same team as the passer that caught the ball
  # Direction          - Where the ball is going during the play
  # Yardage            - Yards gained during the play
  #                      - (Should specify that yardage does not include extra yardage gained from penalties)
  #                      - (Player awarded yardage)
  #                      - (also includes how far kicks have gone during kickoffs and punts)

  # ~ Defensive features ~
  # SoloTackle         - Player awarded a solo tackle from a play
  # AssistedTackle     - Player awarded an assisted tackle from a play
  # SharedTackle       - Player awarded a shared tackle from a play
  # PassDefendedBy     - Defender that defended the passing play
  # PressureBy         - Defender that applied pressure to the passer
  # InterceptedBy      - Defender that intercepted the passing play
  # SackedBy           - Player awarded a sack from a play. (Could be solo or split)
  # ForcedFumbledBy    - Player awarded a forced fumble from a play

  # ~ Unique features (uncommon) ~
  # WhoFumbled         - Player who last held the ball during a fumble.
  # FumbleRecoveredBy  - Player who recovered the fumbled ball
  # FumbleDetails      - A list that has what happened after the fumble
  #                      - [forced fumble by, recovered by, yards gained, tackled by]
  # ReverseDetails     - A list having plays leading up to play reversal
  # InjuredPlayers     - Players that were injured during the play
  # AcceptedPenalty    - Penalty on the field that was accepted
  # DeclinedPenalty    - Penalty on the field that was declined

  # ~ Special teams features ~
  # Kicker             - Player who kicked the ball during a kickoff / punt / extra point / field goal
  # LongSnapper        - Player who snapped the ball during a punt / extra point / field goal
  # Returner           - Player who returned the ball during a kickoff / punt
  # DownedBy           - ? ? ? I forget
  # Holder             - Player who held ball for extra point / field goal
  # BlockedBy          - Player who blocked a punt / extra point / field goal

  new_columns = ["TimeOnTheClock",
                 "PlayType", "Formation", "Passer", "Rusher", "Receiver", "Direction", "Yardage",
                 "SoloTackle", "AssistedTackle", "SharedTackle", 'PassDefendedBy', "PressureBy", "InterceptedBy", "SackedBy", "ForcedFumbleBy",
                 "WhoFumbled", "FumbleRecoveredBy", "FumbleDetails", "ReverseDetails", "InjuredPlayers", "AcceptedPenalty", "DeclinedPenalty",
                 "Kicker", "LongSnapper", "Returner", "DownedBy", "Holder", "BlockedBy"]

  string_columns = ["TimeOnTheClock",
                    "PlayType", "Formation", "Passer", "Rusher", "Receiver", "Direction",
                    "SoloTackle", "AssistedTackle", "SharedTackle", 'PassDefendedBy', "PressureBy", "InterceptedBy", "SackedBy", "ForcedFumbleBy",
                    "WhoFumbled", "FumbleRecoveredBy", "FumbleDetails", "ReverseDetails", "InjuredPlayers", "AcceptedPenalty", "DeclinedPenalty",
                    "Kicker", "LongSnapper", "Returner", "DownedBy", "Holder", "BlockedBy"]

  int_columns = ["Yardage"]

  ########################################
  # RETURN DATAFRAME WITH ADDED FEATURES #
  ########################################

  df_all_plays_cleaned = df_all_plays.copy()
  df_all_plays_cleaned = df_all_plays_cleaned.reindex(columns=df_all_plays_cleaned.columns.tolist() + new_columns)
  df_all_plays_cleaned[string_columns] = df_all_plays_cleaned[string_columns].astype(str)
  df_all_plays_cleaned[int_columns] = df_all_plays_cleaned[int_columns].astype(float)

  #############################################################
  # TRANSFORMING FEATURE VALUES (PREPPING DATA TO BE CLEANED) #
  #############################################################
  df_all_plays_cleaned = playtimeformation_split(df_all_plays_cleaned)
  df_all_plays_cleaned = playstart_split(df_all_plays_cleaned)
  df_all_plays_cleaned = consistent_team_names(df_all_plays_cleaned)

  ########################################
  # GETTING PLAY CATEGORIES AND CLEANING #
  ########################################



  return df_all_plays_cleaned

# TESTING

In [81]:
df_week2_plays_cleaned = clean_dataframe_of_plays(week2_2023_plays)

In [82]:
df_week2_plays_cleaned.shape

(2752, 47)

In [83]:
df_week2_plays_cleaned

Season    Week  Day   Date AwayTeam HomeTeam  Quarter  DriveNumber  \
0       2023  Week 2  THU  09/14      MIN      PHI        1            1   
1       2023  Week 2  THU  09/14      MIN      PHI        1            1   
2       2023  Week 2  THU  09/14      MIN      PHI        1            1   
3       2023  Week 2  THU  09/14      MIN      PHI        1            1   
4       2023  Week 2  THU  09/14      MIN      PHI        1            1   
...      ...     ...  ...    ...      ...      ...      ...          ...   
2747    2023  Week 2  MON  09/18      CLE      PIT        4            6   
2748    2023  Week 2  MON  09/18      CLE      PIT        4            6   
2749    2023  Week 2  MON  09/18      CLE      PIT        4            6   
2750    2023  Week 2  MON  09/18      CLE      PIT        4            7   
2751    2023  Week 2  MON  09/18      CLE      PIT        4            7   

     TeamWithPossession  IsScoringDrive  ...  AcceptedPenalty  \
0                   PHI               1  ...              nan   
1                   PHI               1  ...              nan   
2                   PHI               1  ...              nan   
3                   PHI               1  ...              nan   
4                   PHI               1  ...              nan   
...                 ...             ...  ...              ...   
2747                CLE               0  ...              nan   
2748                CLE               0  ...              nan   
2749                CLE               0  ...              nan   
2750                PIT               0  ...              nan   
2751                PIT               0  ...              nan   

      DeclinedPenalty Kicker LongSnapper Returner DownedBy Holder BlockedBy  \
0                 nan    nan         nan      nan      nan    nan       nan   
1                 nan    nan         nan      nan      nan    nan       nan   
2                 nan    nan         nan      nan      nan    nan       nan   
3                 nan    nan         nan      nan      nan    nan       nan   
4                 nan    nan         nan      nan      nan    nan       nan   
...               ...    ...         ...      ...      ...    ...       ...   
2747              nan    nan         nan      nan      nan    nan       nan   
2748              nan    nan         nan      nan      nan    nan       nan   
2749              nan    nan         nan      nan      nan    nan       nan   
2750              nan    nan         nan      nan      nan    nan       nan   
2751              nan    nan         nan      nan      nan    nan       nan   

     PlayFormation FieldPosition  
0          Kickoff           nan  
1          Shotgun        PHI 25  
2          Shotgun        PHI 31  
3          Shotgun        PHI 30  
4          Shotgun        PHI 37  
...            ...           ...  
2747       Shotgun        PIT 47  
2748       Shotgun        PIT 47  
2749       Shotgun        PIT 49  
2750                          50  
2751                      PIT 49  

[2752 rows x 47 columns]

# PLAYTYPE OBSERVATIONS

## HELPER METHOD

In [84]:
# PURPOSE:
# - A tool that can be used to compare original plays and their cleaned versions

# I would like to return a map that has:
# KEY: index of original unclean play
# VALUE: index(es) of cleaned play

def unclean_to_clean_play_matches(df_unclean_plays, df_clean_plays):

  my_map = {}

  # This list of features is unique to each play
  # - Both the unclean and cleaned versions of the plays have these same features, therefore
  #   they will be used to match unclean plays in 'df_unclean_plays' to clean plays in 'df_clean_plays'
  matching_features = ['Season', 'Week', 'Date', 'AwayTeam', 'HomeTeam', 'Quarter', 'DriveNumber', 'PlayNumberInDrive']

  # Iterate through each row of the unclean plays dataframe
  for u_row in df_unclean_plays.itertuples(index=True):
    u_features = [getattr(u_row, col) for col in matching_features]

    matching_indexes = []
    matches_found = False

    # Iterate through each row of the dataframe of cleaned plays
    # - The starting index will be the index of the unclean play within the main original dataframe of plays
    #   - The matching cleaned pair will either be at the exact same location or higher
    for c_row in df_clean_plays[u_row.Index::].itertuples(index=True):
      c_features = [getattr(c_row, col) for col in matching_features]

      # If a match is found, check for consective rows of matches because some uncleaned plays needed to be cleaned using multiple rows
      # - Once a row that does not match follows one that does, will break the loop because the one play match has been found.
      if u_features == c_features:
        matching_indexes.append(c_row.Index)
        matches_found = True
      elif matches_found:
        my_map[u_row.Index] = matching_indexes
        break

  return my_map

## PASSING PLAYS

In [85]:
# Modifying plays to match cleaned plays transformed features
# ( e.g. Quarter(original) = '1st Quarter
#        Quarter(transform) = 1 )
# - This is needed in order to match plays from the original dataframe
#   to the cleaned dataframe.
df_week2_plays_modified = week2_2023_plays.copy()

df_week2_plays_modified = playtimeformation_split(df_week2_plays_modified)
df_week2_plays_modified = playstart_split(df_week2_plays_modified)
df_week2_plays_modified = consistent_team_names(df_week2_plays_modified)

In [91]:
# All passing plays
df_unclean_pass_plays = df_week2_plays_modified.loc[df_week2_plays_modified['PlayOutcome'].str.contains('Pass')]

map_unclean_clean_pass_plays = unclean_to_clean_play_matches(df_unclean_pass_plays, df_week2_plays_cleaned)

len(map_unclean_clean_pass_plays.keys())

1045

In [92]:
# Every unclean passing play and their associated cleaned play breakdown

for i in map_unclean_clean_pass_plays.keys():
  print(f"({i}, {map_unclean_clean_pass_plays.get(i)})")
  play = df_week2_plays_modified['PlayDescription'].iloc[i]
  play_split = play.split(". ")
  for j in play_split:
    print(j)
  print()

(1, [1])
— J.Hurts pass short right to D.Smith to PHI 31 for 6 yards (B.Murphy; C.Bynum).

(4, [4])
— J.Hurts pass short left to D.Goedert to PHI 36 for -1 yards (C.Bynum).

(5, [5])
— J.Hurts pass deep right to D.Smith to MIN 10 for 54 yards (Th.Jackson).

(6, [6])
— J.Hurts pass short left to A.Brown to MIN 9 for 1 yard (Th.Jackson, C.Bynum).

(11, [11])
— K.Cousins pass deep right to J.Jefferson to MIN 40 for 15 yards (D.Slay).

(12, [12])
— K.Cousins pass short right to J.Jefferson to MIN 41 for 1 yard (D.Slay).

(13, [13])
— K.Cousins pass incomplete short right to A.Mattison [J.Sweat].

(14, [14])
— K.Cousins pass incomplete short middle to K.Osborn [J.Davis].

(16, [16])
— J.Hurts pass incomplete deep right to D.Goedert.

(17, [17])
— J.Hurts pass short right to D.Goedert to PHI 16 for 6 yards (H.Smith).

(25, [25])
— J.Hurts pass short left to D.Goedert to PHI 44 for 1 yard (Th.Jackson, A.Evans).

(29, [29])
— J.Hurts pass short left to A.Brown to MIN 34 for 5 yards (J.Hicks).
